In [1]:
#  https://github.com/medspacy/medspacy

In [1]:
import json
import base64
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console
import psycopg2
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent

In [2]:
conn = _conn_open()
cur = conn.cursor()

OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "localhost" does not exist


In [ ]:


# Load medspacy model
nlp = medspacy.load()
print(nlp.pipe_names)

text = """
Past Medical History:
1. Atrial fibrillation
2. Type II Diabetes Mellitus

Assessment and Plan:
There is no evidence of pneumonia. Continue warfarin for Afib. Follow up for management of type 2 DM.
"""

# Add rules for target concept extraction
target_matcher = nlp.get_pipe("medspacy_target_matcher")
target_rules = [
    TargetRule("atrial fibrillation", "PROBLEM"),
    TargetRule("atrial fibrillation", "PROBLEM", pattern=[{"LOWER": "afib"}]),
    TargetRule("pneumonia", "PROBLEM"),
    TargetRule(
        "Type II Diabetes Mellitus",
        "PROBLEM",
        pattern=[
            {"LOWER": "type"},
            {"LOWER": {"IN": ["2", "ii", "two"]}},
            {"LOWER": {"IN": ["dm", "diabetes"]}},
            {"LOWER": "mellitus", "OP": "?"},
        ],
    ),
    TargetRule("warfarin", "MEDICATION"),
]
target_matcher.add(target_rules)   


doc = nlp(text)


In [ ]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.target_rule, sep="  |  ")
    print()

In [ ]:
visualize_ent(doc)